# Imports

In [1674]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")



# Load datasets

In [1675]:
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')

train_targets_a = pd.read_parquet('../data/A/train_targets.parquet')
train_targets_b = pd.read_parquet('../data/B/train_targets.parquet')
train_targets_c = pd.read_parquet('../data/C/train_targets.parquet')


# Data clean up

In [1676]:

# Data set A, B and C clean up

def data_clean_up(x_train_est, x_train_observe, y_train):

  if 'date_calc' in x_train_est.columns:
    x_train_est.drop(columns="date_calc", inplace=True)

  x_train = pd.concat([x_train_observe, x_train_est])

  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_train.columns[1:]}
  X_train_downscaled = x_train.groupby(x_train.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  y_train.dropna(inplace=True)
  combined_data = pd.merge(X_train_downscaled, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'date_forecast' and 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns="date_forecast", inplace=True)
    combined_data.drop(columns="time", inplace=True)
    combined_data.drop(columns="pv_measurement", inplace=True)


  return combined_data, y_train

x_train_a, y_train_a = data_clean_up(X_train_estimated_a, X_train_observed_a, train_targets_a)
x_train_b, y_train_b = data_clean_up(X_train_estimated_b, X_train_observed_b, train_targets_b)
x_train_c, y_train_c = data_clean_up(X_train_estimated_c, X_train_observed_c, train_targets_c)


def data_clean_up_test(x_test_est):


  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_test_est.columns[1:]}
  X_test_downscaled = x_test_est.groupby(x_test_est.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  if 'date_forecast' in X_test_downscaled.columns:
    X_test_downscaled.drop(columns="date_forecast", inplace=True)

  return X_test_downscaled

X_test_estimated_a = data_clean_up_test(X_test_estimated_a)
X_test_estimated_b = data_clean_up_test(X_test_estimated_b)
X_test_estimated_c = data_clean_up_test(X_test_estimated_c)


# Feature engineering

In [1677]:
# Do feature selection etc.

# Polynomial features of degree 2 of most important features

def polynomial_feature(x_dataset, features):
  
  for feature in features:
    x_dataset[feature + ':squared'] = x_dataset[feature] ** 2
    x_dataset[feature + ':cubed'] = x_dataset[feature] ** 3
    x_dataset[feature + ':sqrt'] = np.sqrt(x_dataset[feature])

  return x_dataset


# x_train_a = polynomial_feature(x_train_a, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# x_train_b = polynomial_feature(x_train_b, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# x_train_c = polynomial_feature(x_train_c, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])

# X_test_estimated_a = polynomial_feature(X_test_estimated_a, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# X_test_estimated_b = polynomial_feature(X_test_estimated_b, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])
# X_test_estimated_c = polynomial_feature(X_test_estimated_c, ['direct_rad:W', 'diffuse_rad:W', 'sun_elevation:d'])

def drop_features(x_dataset, features):
  for feature in features:
    x_dataset.drop(columns=feature, inplace=True)

  return x_dataset

# x_train_a = drop_features(x_train_a, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# x_train_b = drop_features(x_train_b, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# x_train_c = drop_features(x_train_c, ['snow_density:kgm3', 'fresh_snow_3h:cm'])

# X_test_estimated_a = drop_features(X_test_estimated_a, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# X_test_estimated_b = drop_features(X_test_estimated_b, ['snow_density:kgm3', 'fresh_snow_3h:cm'])
# X_test_estimated_c = drop_features(X_test_estimated_c, ['snow_density:kgm3', 'fresh_snow_3h:cm'])



# Training the model

In [1678]:

# Split the data into training and validation sets

x_train_a, x_val_a, y_train_a, y_val_a = train_test_split(x_train_a, y_train_a, test_size=0.17, random_state=42)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(x_train_b, y_train_b, test_size=0.17, random_state=42)
x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(x_train_c, y_train_c, test_size=0.17, random_state=42)

model_a = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.25, max_depth=10, min_child_weight=2, gamma=150, reg_lambda=20)
model_b = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.25, max_depth=10, min_child_weight=4, gamma=34, reg_lambda=20)
model_c = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, learning_rate=0.24, max_depth=10, min_child_weight=3, gamma=8, reg_lambda=44)
# max_depth = 6 gives best


model_a.fit(x_train_a, y_train_a)
model_b.fit(x_train_b, y_train_b)
model_c.fit(x_train_c, y_train_c)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=8, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.24, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# Make predictions

In [1679]:


y_pred_a = model_a.predict(X_test_estimated_a)
y_pred_b = model_b.predict(X_test_estimated_b)
y_pred_c = model_c.predict(X_test_estimated_c)

y_pred = np.concatenate((y_pred_a, y_pred_b, y_pred_c), axis=0)

# Evaluate prediction

In [1680]:
# Evaluate the model based on the validation data

mse_a = mean_squared_error(y_val_a, model_a.predict(x_val_a))
print("MSE for A: ", mse_a)
mse_b = mean_squared_error(y_val_b, model_b.predict(x_val_b))
print("MSE for B: ", mse_b)
mse_c = mean_squared_error(y_val_c, model_c.predict(x_val_c))
print("MSE for C: ", mse_c)
print("Mean MSE: ", (mse_a + mse_b + mse_c) / 3)

# Evaluate the predictions

score_a = model_a.score(x_val_a, y_val_a)
score_b = model_b.score(x_val_b, y_val_b)
score_c = model_c.score(x_val_c, y_val_c)

print("Score A: ", score_a)
print("Score B: ", score_b)
print("Score C: ", score_c)

# Get feature importance scores
model_a.fit(x_train_a, y_train_a)
feature_importance_scores = model_a.feature_importances_

# Create a DataFrame to associate features with their importance scores
feature_importance_df = pd.DataFrame({'Feature': x_train_a.columns, 'Importance': feature_importance_scores})

# Sort features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importance scores
print(feature_importance_df)


MSE for A:  162445.32803435044
MSE for B:  4843.660688571223
MSE for C:  2423.0632593393907
Mean MSE:  56570.68399408702
Score A:  0.8817545740855711
Score B:  0.8742617169876488
Score C:  0.9188062828577899
                           Feature  Importance
36                 sun_elevation:d    0.740617
4                  clear_sky_rad:W    0.104682
10                    direct_rad:W    0.027994
11                 direct_rad_1h:J    0.023406
30               snow_density:kgm3    0.018142
16               fresh_snow_24h:cm    0.011605
38                     t_1000hPa:K    0.006030
23            precip_type_5min:idx    0.005982
27                 rain_water:kgm2    0.004892
24               pressure_100m:hPa    0.004855
7                   dew_point_2m:K    0.004421
14               fresh_snow_12h:cm    0.004297
22                  precip_5min:mm    0.003959
3            clear_sky_energy_1h:J    0.002940
29                sfc_pressure:hPa    0.002762
21                msl_pressure:hPa    0.

# Create submission

In [1681]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission.csv', index=False)